## Clean up
Please make sure to comment the below section if you are planning to use the Knowledge Base that you created above for building your RAG application.
If you only wanted to try out creating the KB using SDK, then please make sure to delete all the resources that were created as you will be incurred cost for storing documents in OSS index.

#### Delete KnowledgeBase and delete resources after completing all the notebooks.


In [1]:
%store -r

In [2]:
import boto3
import time

boto3_session = boto3.Session()
bedrock_agent_client = boto3_session.client('bedrock-agent', region_name=boto3_session.region_name)
aoss_client = boto3.client('opensearchserverless')
s3_client = boto3_session.client('s3', region_name=boto3_session.region_name)
iam_client = boto3.client("iam")

#### Delete Bedrock KnowledgeBase data sources

In [3]:
# fetch data source(s)associated with kb
response = bedrock_agent_client.list_data_sources(knowledgeBaseId=kb_id)
data_sources_list = [ds_summary for ds_summary in response['dataSourceSummaries']]

for idx, ds in enumerate(data_sources_list):
    bedrock_agent_client.delete_data_source(dataSourceId = ds["dataSourceId"], knowledgeBaseId=ds["knowledgeBaseId"])
    time.sleep(10)


#### Remove KnowledgeBases and OpenSearch Collection

In [4]:
# Fetch kb execution role
response = bedrock_agent_client.get_knowledge_base(knowledgeBaseId=kb_id)
kb_role_name = response['knowledgeBase']['roleArn'].split("/")[-1]
print(f"The role name of the knowledge base is {kb_role_name}")
# Fetch the role ARN

The role name of the knowledge base is AmazonBedrockExecutionRoleForKnowledgeBase_712


In [5]:
# fetch all attched policies with kb execution role
kb_attached_role_policies_response = iam_client.list_attached_role_policies(
    RoleName=kb_role_name)
kb_attached_role_policies = kb_attached_role_policies_response['AttachedPolicies']
print(f"The attached policies of the role {kb_role_name} are {kb_attached_role_policies}")

The attached policies of the role AmazonBedrockExecutionRoleForKnowledgeBase_712 are [{'PolicyName': 'AmazonBedrockOSSPolicyForKnowledgeBase_712', 'PolicyArn': 'arn:aws:iam::183295442349:policy/AmazonBedrockOSSPolicyForKnowledgeBase_712'}, {'PolicyName': 'AmazonBedrockS3PolicyForKnowledgeBase_712', 'PolicyArn': 'arn:aws:iam::183295442349:policy/AmazonBedrockS3PolicyForKnowledgeBase_712'}, {'PolicyName': 'AmazonBedrockFoundationModelPolicyForKnowledgeBase_712', 'PolicyArn': 'arn:aws:iam::183295442349:policy/AmazonBedrockFoundationModelPolicyForKnowledgeBase_712'}]


In [6]:
bedrock_agent_client.delete_knowledge_base(knowledgeBaseId=kb_id)
aoss_client.delete_collection(id=collection['createCollectionDetail']['id'])
aoss_client.delete_access_policy(type="data", name=access_policy['accessPolicyDetail']['name'])
aoss_client.delete_security_policy(type="network", name=network_policy['securityPolicyDetail']['name'])
aoss_client.delete_security_policy(type="encryption", name=encryption_policy['securityPolicyDetail']['name'])

{'ResponseMetadata': {'RequestId': '121df44c-b191-4134-9065-060a16660a7c',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '121df44c-b191-4134-9065-060a16660a7c',
   'date': 'Tue, 22 Apr 2025 16:49:09 GMT',
   'content-type': 'application/x-amz-json-1.0',
   'content-length': '2',
   'connection': 'keep-alive'},
  'RetryAttempts': 0}}

#### Delete role and policies


In [7]:
print(kb_attached_role_policies)

[{'PolicyName': 'AmazonBedrockOSSPolicyForKnowledgeBase_712', 'PolicyArn': 'arn:aws:iam::183295442349:policy/AmazonBedrockOSSPolicyForKnowledgeBase_712'}, {'PolicyName': 'AmazonBedrockS3PolicyForKnowledgeBase_712', 'PolicyArn': 'arn:aws:iam::183295442349:policy/AmazonBedrockS3PolicyForKnowledgeBase_712'}, {'PolicyName': 'AmazonBedrockFoundationModelPolicyForKnowledgeBase_712', 'PolicyArn': 'arn:aws:iam::183295442349:policy/AmazonBedrockFoundationModelPolicyForKnowledgeBase_712'}]


In [8]:
for policy in kb_attached_role_policies:
    # If policy is already detached, skip
        iam_client.detach_role_policy(
            RoleName=kb_role_name,
            PolicyArn=policy['PolicyArn']
        )
        time.sleep(10)
        #delete the policy
        iam_client.delete_policy(
            PolicyArn=policy['PolicyArn']
        )
        

In [9]:
# List all inline policies attached to the role (This is not in the default script!!!)
inline_policies_response = iam_client.list_role_policies(RoleName=kb_role_name)
inline_policies = inline_policies_response['PolicyNames']

# Delete all inline policies
for policy_name in inline_policies:
    iam_client.delete_role_policy(
        RoleName=kb_role_name,
        PolicyName=policy_name
    )
    print(f"Deleted inline policy: {policy_name}")

Deleted inline policy: BedrockInvokeModelPolicy


In [10]:
# Delete the IAM role
iam_client.delete_role(RoleName=kb_role_name)

{'ResponseMetadata': {'RequestId': '368d5052-ae9d-48ed-9e3c-a137b54aa012',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'date': 'Tue, 22 Apr 2025 16:49:42 GMT',
   'x-amzn-requestid': '368d5052-ae9d-48ed-9e3c-a137b54aa012',
   'content-type': 'text/xml',
   'content-length': '200'},
  'RetryAttempts': 0}}

In [11]:
# for policy in kb_attached_role_policies:
#     iam_client.delete_policy(PolicyArn=policy['PolicyArn'])
# USELESS as the policy is already deleted as the needed condition for the role to be deleted

#### Delete S3 objects

In [12]:
objects = s3_client.list_objects(Bucket=bucket_name)
if 'Contents' in objects:
    for obj in objects['Contents']:
        s3_client.delete_object(Bucket=bucket_name, Key=obj['Key'])
s3_client.delete_bucket(Bucket=bucket_name)

{'ResponseMetadata': {'RequestId': 'CQ1GG4R3A3JMQ85X',
  'HostId': 'ZoJA4XG8YIbhaDBAJxs7FFLt5hSzKG/GsCB+7pOHPYtY178MUd0RPUcos+GH7PymS9pnc+e5XZfnXKfLX3w2fsVQr4aAArqq',
  'HTTPStatusCode': 204,
  'HTTPHeaders': {'x-amz-id-2': 'ZoJA4XG8YIbhaDBAJxs7FFLt5hSzKG/GsCB+7pOHPYtY178MUd0RPUcos+GH7PymS9pnc+e5XZfnXKfLX3w2fsVQr4aAArqq',
   'x-amz-request-id': 'CQ1GG4R3A3JMQ85X',
   'date': 'Tue, 22 Apr 2025 16:49:45 GMT',
   'server': 'AmazonS3'},
  'RetryAttempts': 0}}